# <font color=yellow>[1] 사람들이 한 대사를 데이터 분석을 해서, 단어들간에 어떤 연관성이 있는지 실습을 하겠음</font>

- 실생활과 유사한 대화형식
- 이 사람들이 말한 단어들 위주로 학습이 됨

In [1]:
import pandas as pd
from collections import Counter

df = pd.read_csv(r'.\simpsons_dataset.csv')
df.shape

(158314, 2)

In [2]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
counts = Counter(df['raw_character_text'])
counts.most_common()

[('Homer Simpson', 29782),
 (nan, 17814),
 ('Marge Simpson', 14141),
 ('Bart Simpson', 13759),
 ('Lisa Simpson', 11489),
 ('C. Montgomery Burns', 3162),
 ('Moe Szyslak', 2862),
 ('Seymour Skinner', 2438),
 ('Ned Flanders', 2144),
 ('Grampa Simpson', 1880),
 ('Milhouse Van Houten', 1862),
 ('Chief Wiggum', 1830),
 ('Krusty the Clown', 1768),
 ('Nelson Muntz', 1172),
 ('Lenny Leonard', 1166),
 ('Apu Nahasapeemapetilon', 1006),
 ('Waylon Smithers', 996),
 ('Kent Brockman', 891),
 ('Carl Carlson', 883),
 ('Edna Krabappel-Flanders', 739),
 ('Dr. Julius Hibbert', 691),
 ('Barney Gumble', 611),
 ('Selma Bouvier', 611),
 ('Sideshow Bob', 576),
 ('Rev. Timothy Lovejoy', 558),
 ('Crowd', 540),
 ('Groundskeeper Willie', 534),
 ('Gary Chalmers', 523),
 ('Ralph Wiggum', 507),
 ('Mayor Joe Quimby', 503),
 ('Patty Bouvier', 479),
 ('Comic Book Guy', 478),
 ('Otto Mann', 423),
 ('Martin Prince', 409),
 ('Announcer', 387),
 ('Kids', 365),
 ('Jimbo Jones', 357),
 ('Sideshow Mel', 352),
 ('Lou', 350),
 (

# <font color=yellow>[2] 모델들이 질문들의 의미를 잘 파악하고, 같은 것으로 묶어줄 수 있는지에 대한 실습을 할 것임</font>
- Quora에서는 같은 의미를 가진 질문들을 이런식으로 하나로 묶어서 display를 해줌
- 관련해서 단 답변들을 확인할 수 있음

### <font color=green>질문 두개가 있고, 두개가 서로 duplicate인 것을 잘 묶는지 testing하는 것이 목적</font>

In [4]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("quora", trust_remote_code=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['questions', 'is_duplicate'],
        num_rows: 404290
    })
})

In [5]:
raw_df = dataset["train"].to_pandas()
raw_df

,questions,is_duplicate
0,"{'id': [1, 2], 'text': ['What is the step by s...",False
1,"{'id': [3, 4], 'text': ['What is the story of ...",False
2,"{'id': [5, 6], 'text': ['How can I increase th...",False
3,"{'id': [7, 8], 'text': ['Why am I mentally ver...",False
4,"{'id': [9, 10], 'text': ['Which one dissolve i...",False
...,...,...
404285,"{'id': [433578, 379845], 'text': ['How many ke...",False
404286,"{'id': [18840, 155606], 'text': ['Do you belie...",True
404287,"{'id': [537928, 537929], 'text': ['What is one...",False
404288,"{'id': [537930, 537931], 'text': ['What is the...",False


In [6]:
raw_df.loc[0]['questions']

{'id': array([1, 2]),
 'text': array(['What is the step by step guide to invest in share market in india?',
        'What is the step by step guide to invest in share market?'],
       dtype=object)}

In [7]:
raw_df = raw_df.loc[raw_df['is_duplicate']==True].reset_index(drop=True)
raw_df

,questions,is_duplicate
0,"{'id': [11, 12], 'text': ['Astrology: I am a C...",True
1,"{'id': [15, 16], 'text': ['How can I be a good...",True
2,"{'id': [23, 24], 'text': ['How do I read and f...",True
3,"{'id': [25, 26], 'text': ['What can make Physi...",True
4,"{'id': [27, 28], 'text': ['What was your first...",True
...,...,...
149258,"{'id': [537922, 537923], 'text': ['What are so...",True
149259,"{'id': [99131, 81495], 'text': ['Why is Manaph...",True
149260,"{'id': [1931, 16773], 'text': ['How does a lon...",True
149261,"{'id': [537926, 537927], 'text': ['What does J...",True


In [8]:
raw_df.loc[0, 'questions']

{'id': array([11, 12]),
 'text': array(['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
        "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"],
       dtype=object)}

In [9]:
# 중복되는 id를 개별 컬럼으로 배치
raw_df["q1"] = raw_df["questions"].apply(lambda q: q["text"][0])
raw_df["q2"] = raw_df["questions"].apply(lambda q: q["text"][1])
raw_df["id1"] = raw_df["questions"].apply(lambda q: q["id"][0])
raw_df["id2"] = raw_df["questions"].apply(lambda q: q["id"][1])
raw_df

,questions,is_duplicate,q1,q2,id1,id2
0,"{'id': [11, 12], 'text': ['Astrology: I am a C...",True,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",11,12
1,"{'id': [15, 16], 'text': ['How can I be a good...",True,How can I be a good geologist?,What should I do to be a great geologist?,15,16
2,"{'id': [23, 24], 'text': ['How do I read and f...",True,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,23,24
3,"{'id': [25, 26], 'text': ['What can make Physi...",True,What can make Physics easy to learn?,How can you make physics easy to learn?,25,26
4,"{'id': [27, 28], 'text': ['What was your first...",True,What was your first sexual experience like?,What was your first sexual experience?,27,28
...,...,...,...,...,...,...
149258,"{'id': [537922, 537923], 'text': ['What are so...",True,What are some outfit ideas to wear to a frat p...,What are some outfit ideas wear to a frat them...,537922,537923
149259,"{'id': [99131, 81495], 'text': ['Why is Manaph...",True,Why is Manaphy childish in Pokémon Ranger and ...,Why is Manaphy annoying in Pokemon ranger and ...,99131,81495
149260,"{'id': [1931, 16773], 'text': ['How does a lon...",True,How does a long distance relationship work?,How are long distance relationships maintained?,1931,16773
149261,"{'id': [537926, 537927], 'text': ['What does J...",True,What does Jainism say about homosexuality?,What does Jainism say about Gays and Homosexua...,537926,537927


In [10]:
q1_to_q2 = raw_df.copy().rename(columns={"q1": "text", "id1": "id", "id2": "dq_id"}).drop(columns=["questions", "q2"])
q2_to_q1 = raw_df.copy().rename(columns={"q2": "text", "id2": "id", "id1": "dq_id"}).drop(columns=["questions", "q1"])
q1_to_q2

,is_duplicate,text,id,dq_id
0,True,Astrology: I am a Capricorn Sun Cap moon and c...,11,12
1,True,How can I be a good geologist?,15,16
2,True,How do I read and find my YouTube comments?,23,24
3,True,What can make Physics easy to learn?,25,26
4,True,What was your first sexual experience like?,27,28
...,...,...,...,...
149258,True,What are some outfit ideas to wear to a frat p...,537922,537923
149259,True,Why is Manaphy childish in Pokémon Ranger and ...,99131,81495
149260,True,How does a long distance relationship work?,1931,16773
149261,True,What does Jainism say about homosexuality?,537926,537927


In [11]:
flat_df = pd.concat([q1_to_q2, q2_to_q1])
flat_df

,is_duplicate,text,id,dq_id
0,True,Astrology: I am a Capricorn Sun Cap moon and c...,11,12
1,True,How can I be a good geologist?,15,16
2,True,How do I read and find my YouTube comments?,23,24
3,True,What can make Physics easy to learn?,25,26
4,True,What was your first sexual experience like?,27,28
...,...,...,...,...
149258,True,What are some outfit ideas wear to a frat them...,537923,537922
149259,True,Why is Manaphy annoying in Pokemon ranger and ...,81495,99131
149260,True,How are long distance relationships maintained?,16773,1931
149261,True,What does Jainism say about Gays and Homosexua...,537927,537926


In [12]:
flat_df = flat_df.sort_values(by=['id']).reset_index(drop=True)
flat_df

,is_duplicate,text,id,dq_id
0,True,Astrology: I am a Capricorn Sun Cap moon and c...,11,12
1,True,"I'm a triple Capricorn (Sun, Moon and ascendan...",12,11
2,True,How can I be a good geologist?,15,16
3,True,What should I do to be a great geologist?,16,15
4,True,How do I read and find my YouTube comments?,23,24
...,...,...,...,...
298521,True,What would happen if I put milk instead of wat...,537916,537915
298522,True,What are some outfit ideas to wear to a frat p...,537922,537923
298523,True,What are some outfit ideas wear to a frat them...,537923,537922
298524,True,What does Jainism say about homosexuality?,537926,537927


<font color=green>id가 11번이라도 duplicate 질문이 여러개일 수도 있음</font>

In [13]:
flat_df.loc[flat_df['id']==568]

,is_duplicate,text,id,dq_id
1263,True,How can I make money online with free of cost?,568,569
1264,True,How can I make money online with free of cost?,568,214223
1265,True,How can I make money online with free of cost?,568,8268
1266,True,How can I make money online with free of cost?,568,5511
1267,True,How can I make money online with free of cost?,568,36751
1268,True,How can I make money online with free of cost?,568,101583
1269,True,How can I make money online with free of cost?,568,92406


<font color=green>일부 샘플만 활용</font>

In [14]:
flat_df = flat_df.loc[((flat_df['id'] <= 15000) & (flat_df['dq_id'] <= 15000))]
flat_df

,is_duplicate,text,id,dq_id
0,True,Astrology: I am a Capricorn Sun Cap moon and c...,11,12
1,True,"I'm a triple Capricorn (Sun, Moon and ascendan...",12,11
2,True,How can I be a good geologist?,15,16
3,True,What should I do to be a great geologist?,16,15
4,True,How do I read and find my YouTube comments?,23,24
...,...,...,...,...
33163,True,When a question on Quora is marked as 'needing...,14999,966
33165,True,When a question on Quora is marked as 'needing...,14999,2929
33169,True,When a question on Quora is marked as 'needing...,14999,2930
33173,True,When a question on Quora is marked as 'needing...,14999,11797


In [15]:
flat_df.shape

(12574, 4)

<font color=green>각 질문 하나당 중복되는 질문 id를 list 형태로 저장</font>

In [16]:
df = flat_df.drop_duplicates("id")
df

,is_duplicate,text,id,dq_id
0,True,Astrology: I am a Capricorn Sun Cap moon and c...,11,12
1,True,"I'm a triple Capricorn (Sun, Moon and ascendan...",12,11
2,True,How can I be a good geologist?,15,16
3,True,What should I do to be a great geologist?,16,15
4,True,How do I read and find my YouTube comments?,23,24
...,...,...,...,...
33147,True,How do girl feel after losing virginity?,14989,14988
33152,True,What is it like to lose 30 pounds in one month?,14992,9736
33160,True,Why is 2000 rupee note released earlier than 5...,14997,14998
33161,True,Why does RBI release 2000 note first instead o...,14998,14997


<font color=green>df["id"] (duplicate이 제거된) 에서 원소를 하나씩 가져와 flat_df["id"]==qid인것을 뽑고 (중복인 것을 가져옴) list로 바꿈</font>

- df에 이어 붙여줌

In [17]:
df.loc[:, "duplicated_questions"] = df["id"].apply(lambda qid: flat_df[flat_df["id"] == qid]["dq_id"].tolist())
df

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_13900\2645713361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "duplicated_questions"] = df["id"].apply(lambda qid: flat_df[flat_df["id"] == qid]["dq_id"].tolist())


,is_duplicate,text,id,dq_id,duplicated_questions
0,True,Astrology: I am a Capricorn Sun Cap moon and c...,11,12,[12]
1,True,"I'm a triple Capricorn (Sun, Moon and ascendan...",12,11,[11]
2,True,How can I be a good geologist?,15,16,[16]
3,True,What should I do to be a great geologist?,16,15,[15]
4,True,How do I read and find my YouTube comments?,23,24,[24]
...,...,...,...,...,...
33147,True,How do girl feel after losing virginity?,14989,14988,[14988]
33152,True,What is it like to lose 30 pounds in one month?,14992,9736,"[9736, 9737]"
33160,True,Why is 2000 rupee note released earlier than 5...,14997,14998,[14998]
33161,True,Why does RBI release 2000 note first instead o...,14998,14997,[14997]


<font color=green>우리가 활용할 최종 데이터 포맷</font>

In [18]:
df = df.drop(columns=["dq_id", "is_duplicate"])
df.loc[:, 'length'] = [len(x) for x in df['duplicated_questions']]
df

,text,id,duplicated_questions,length
0,Astrology: I am a Capricorn Sun Cap moon and c...,11,[12],1
1,"I'm a triple Capricorn (Sun, Moon and ascendan...",12,[11],1
2,How can I be a good geologist?,15,[16],1
3,What should I do to be a great geologist?,16,[15],1
4,How do I read and find my YouTube comments?,23,[24],1
...,...,...,...,...
33147,How do girl feel after losing virginity?,14989,[14988],1
33152,What is it like to lose 30 pounds in one month?,14992,"[9736, 9737]",2
33160,Why is 2000 rupee note released earlier than 5...,14997,[14998],1
33161,Why does RBI release 2000 note first instead o...,14998,[14997],1


In [19]:
df.loc[[len(i)>2 for i in df.duplicated_questions]]

,text,id,duplicated_questions,length
14,What would a Trump presidency mean for current...,31,"[6937, 12544, 11435, 32, 1101]",5
24,How will a Trump presidency affect the student...,32,"[2067, 1100, 6937, 12544, 31, 1101, 2066, 1143...",10
46,Why are so many Quora users posting questions ...,37,"[12639, 1358, 4951, 1357, 6551, 38]",6
63,Why do people ask Quora questions which can be...,38,"[4950, 4407, 4408, 6552, 6551, 12638, 5041, 12...",14
126,What is best way to make money online?,57,"[6800, 12851, 13144, 6099, 4038, 8037, 6799, 1...",23
...,...,...,...,...
33034,Where can you find out what needs to be improv...,14958,"[14288, 967, 966, 2929]",4
33047,Which is the best way to learn hacking just as...,14962,"[8240, 14963, 14168, 14167, 8401, 8400, 8241]",7
33080,What are the safety precautions on handling sh...,14966,"[1596, 10671, 12719, 8119, 5903, 5434, 1595, 1...",10
33112,How did the 2016 US election polls get it so w...,14976,"[14977, 10435, 10434]",3


In [ ]:
df.to_csv("quora_dataset.csv")

# <font color=yellow>[3] 뉴스 헤드라인을 임베딩 벡터로 변환하고, 다른 머신러닝 모델들이랑 결합을 해서 어떻게 데이터를 창의적으로 clustering하고 제외시키는지 실습을 진행</font>

In [20]:
df = pd.read_csv(r".\abcnews.csv")
df

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1244179,20211231,two aged care residents die as state records 2...
1244180,20211231,victoria records 5;919 new cases and seven deaths
1244181,20211231,wa delays adopting new close contact definition
1244182,20211231,western ringtail possums found badly dehydrate...


<font color=green>우리는 데이터셋을 한달치만 활용</font>

In [21]:
df.publish_date.max(), df.publish_date.min()

(20211231, 20030219)

In [22]:
news_2020 = df.loc[(df['publish_date']>=20200101) & (df['publish_date']<20200201)].reset_index(drop=True)
news_2020

,publish_date,headline_text
0,20200101,a new type of resolution for the new year
1,20200101,adelaide records driest year in more than a de...
2,20200101,adelaide riverbank catches alight after new ye...
3,20200101,adelaides 9pm fireworks spark blaze on riverbank
4,20200101,archaic legislation governing nt women propert...
...,...,...
2442,20200131,who coronavirus global emergency
2443,20200131,who declares coronavirus outbreak as global he...
2444,20200131,will travel insurance cover trip cancelled ove...
2445,20200131,world youngest leader 33 years old offers hope...


In [23]:
news_2020.to_csv("abcnews_2020.csv", index=False)

# <font color=yellow>[4] 이력서 데이터: 특정화 되어있지 않은 문서에서, 공통 정보 추출, 특정한 형식으로 전처리 후 서치 </font>

- 누가 쓰느냐에 따라 포멧이 전부 다름
- 그러나 안에 들어있는 정보는 유사함

In [24]:
resume = pd.read_csv(r".\Resume.csv")
resume

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [25]:
resume.Category.unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)